In [1]:
import numpy as np
import pandas as pd


from environment import EpsilonGreedyEnvironment
from services import DatabaseService

from services.database import ActivityClusterDatabaseService,ActivityUserDatabaseService,ClusterFeatureDatabaseService
# from subscribers import new_activity_callback, get_feed_callback, record_result_callback

In [2]:
activity_cluster = pd.DataFrame.from_records(ActivityClusterDatabaseService().get_all()).drop('score',axis=1)
activity_cluster

Connected to Database Service


,cluster,feature,activityId
0,37,dishware,5dc0607a53d7270024cee005
1,37,jeans,61b344f01f46d2d2cbdffbfe
2,149,black,60cb52366c337f51c2830335
3,405,light,61880657b9fba8a46f75e250
4,676,myvibe,616def40bb137ccf44322bb3
...,...,...,...
95276,37,ingredient,5e25a68455bd600024a0666d
95277,461,tableware,614448fd11d1927c26c6fa55
95278,194,food,5fd2174b9cf1fdb7daf5af90
95279,37,grey,60c4b852f1be8caa96ed531a


In [3]:
cluster_feature = pd.DataFrame.from_records(ClusterFeatureDatabaseService().get_all())
clusters = cluster_feature['cluster'].unique().tolist()

Connected to Database Service


In [4]:
len(clusters)

1132

In [5]:
activity_user = pd.DataFrame.from_records(ActivityUserDatabaseService().get_all())
activity_user

Connected to Database Service


,activityId,userId,score
0,5cc51c9f3a614e0024005239,5cc51a0b3a614e0024005237,0.2
1,5cc51c9f3a614e0024005239,5cc70cf80c2e8f002494034a,0.2
2,5cc51c9f3a614e0024005239,5cc875aaa130ee002484131b,0.2
3,5cc51c9f3a614e0024005239,5d093a52cf30860024f61a3f,0.2
4,5cc51c9f3a614e0024005239,5d1b5d64e045290024275e99,0.2
...,...,...,...
212878,62accc5a4a194b6ff4e31243,62a76837c4f892b9842df1f0,0.2
212879,62ad5e5b4a194b6ff4e31db8,61b9f769abb948fa31191ecb,0.2
212880,62adb5fa4a194b6ff4e33ed4,62755cb7126cfb106f828f5b,0.2
212881,62adc0694a194b6ff4e341f9,616fbe1c0d67f630e59b5bad,0.2


In [6]:
cluster_userid_score = activity_user.merge(activity_cluster,on="activityId").groupby(["cluster","userId"]).mean().reset_index()
cluster_userid_score

,cluster,userId,score
0,0,5f990dc6ac37d7d071304fc7,0.2
1,0,5f9fec77761cfd73158a457f,0.2
2,0,5f9fec78761cfd73158a4581,0.2
3,0,5f9fec78761cfd73158a4582,0.2
4,0,5f9fec78761cfd73158a4583,0.2
...,...,...,...
194432,1131,5f990dc6ac37d7d071304fc7,0.2
194433,1131,5f9fec77761cfd73158a457f,0.2
194434,1131,5f9fec78761cfd73158a4581,0.2
194435,1131,5f9fec78761cfd73158a4582,0.2


In [7]:
epsilon = 0.4
item_col_name = 'cluster'
visitor_col_name = 'userId'

env = EpsilonGreedyEnvironment(
    epsilon, cluster_userid_score, clusters, item_col_name, visitor_col_name,)


In [8]:
a,b,c = env.get_state()

In [9]:
len(a)==len(b)==len(c)

True

In [10]:
cluster_userid_score

,cluster,userId,score
0,0,5f990dc6ac37d7d071304fc7,0.2
1,0,5f9fec77761cfd73158a457f,0.2
2,0,5f9fec78761cfd73158a4581,0.2
3,0,5f9fec78761cfd73158a4582,0.2
4,0,5f9fec78761cfd73158a4583,0.2
...,...,...,...
194432,1131,5f990dc6ac37d7d071304fc7,0.2
194433,1131,5f9fec77761cfd73158a457f,0.2
194434,1131,5f9fec78761cfd73158a4581,0.2
194435,1131,5f9fec78761cfd73158a4582,0.2


In [12]:
df = cluster_userid_score.reset_index()

for index, row in df.iterrows():
    cluster_idx = a.index(row['cluster'])
    env.record_result(None,cluster_idx,row['score'])

In [13]:
a,b,c = env.get_state()

In [69]:
env.select_item()

29

In [70]:
env.save_state()